# REFINITIV EIKON - TRADE AND QUOTES 

In [ ]:
import time
import eikon as ek
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import refinitiv.data as rd
rd.open_session()

# SET VARIABLES

In [ ]:
# GERMAN MARKET SPECIFIC
index = "0#.GDAXI"
home_id = "d."

In [ ]:
# SET MONTH
month = "X"
year = "20XX"
month_year = month + " " + year

In [ ]:
# WHICH MTFs
biggest_mtfs = [".DE", ".DXE", ".TQE", ".AQE"]

# SET APP KEY

In [ ]:
app_key = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

In [ ]:
ek.set_app_key(app_key)

In [ ]:
ek.set_timeout(1e6)

# MAKE AND SET PATH TO SAVE

In [ ]:
folder_name = Path(f"TAQ_{year}_{month}")
folder_name.mkdir(parents=True, exist_ok=True)
path_to_save = f"./TAQ_{year}_{month}/"

# GET CONSTITUENTS

In [ ]:
index_constituents = pd.read_pickle("index_constituents")

# GET FRAGMENTATION

In [ ]:
fragmentation = pd.read_pickle("fragmentation")

In [ ]:
constituents = fragmentation["ric"].unique()
i = 0
for con in constituents:
    print(f" [{i}] {con}")
    i += 1

# GET TAQ DATA

In [ ]:
start_date_dt = datetime.strptime(month_year, '%m %Y')
start_date = start_date_dt.strftime('%Y-%m-%d')
end_date_dt = start_date_dt + relativedelta(days = 30)
end_date = end_date_dt.strftime('%Y-%m-%d')

In [ ]:
tic = time.time()
counter = 0
df_month = pd.DataFrame()

for con in constituents:
    counter += 1
    print(f"Start {counter}: {con}")
    df_con = pd.DataFrame()
    fragmentation_con = list(fragmentation.loc[fragmentation["ric"] == con, "fragmentation"])
    
    for ric in fragmentation_con:
        df_ric = rd.get_history(ric, start = start_date, end = end_date, interval = '1min')
        df_ric["RIC"] = ric
        df_con = pd.concat([df_con, df_ric], ignore_index = False)
        print(f"     Done {ric}")
    
    df_month = pd.concat([df_month, df_con], ignore_index = False)
    # Set hashtag before code below if you do not want 1 file per constituent
    df_con.to_csv(path_to_save + "TAQ_" + con + "_" + year + "_" + month +  ".csv")  
    
    toc = time.time() 
    print(f"Done {con} in {round(toc-tic,0)} seconds")

# Set hashtag before code below if you do not want 1 file with all data in it
df_month.to_csv(path_to_save + "TAQ_" + year + "_" + month +  ".csv")